# Code 3

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Kosakata sederhana untuk contoh
src_vocab = {'How': 0, 'are': 1, 'you': 2, '?': 3, '<pad>': 4, '<sos>': 5, '<eos>': 6}
trg_vocab = {'comment': 0, 'vas': 1, 'tu': 2, '?': 3, '<pad>': 4, '<sos>': 5, '<eos>': 6}

src_vocab_size = len(src_vocab)
trg_vocab_size = len(trg_vocab)

# Membuat data dummy
src_data = torch.tensor([[5, 0, 1, 2, 3, 6]])  # Input: "<sos> Nice to meet you <eos>"
trg_data = torch.tensor([[5, 0, 1, 2, 3, 6]])  # Target: "<sos> ravi de vous rencontrer <eos>"

# Dataset dan DataLoader
train_dataset = TensorDataset(src_data, trg_data)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [3]:
# Model Encoder-Decoder
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(1)
        embedded = self.embedding(x)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size)

        hidden, cell = self.encoder(src)

        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t] = output

            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

Kode yang dibangun mendefinisikan model **Encoder-Decoder** dengan LSTM untuk tugas pemrosesan sekuensial seperti penerjemahan sebuah kata dengan data yang telah ddi inputkan. **Encoder** memproses input sekuensial melalui embedding dan LSTM untuk menghasilkan hidden state dan cell state. **Decoder** menerima state ini dan menghasilkan prediksi token satu per satu, menggunakan embedding, LSTM, dan layer linear untuk memetakan ke ruang output.

Kelas **Seq2Seq** menggabungkan encoder dan decoder, dan menggunakan **teacher forcing** untuk menentukan apakah input berikutnya ke decoder adalah token target sebenarnya atau prediksi sebelumnya. Hasil akhirnya adalah tensor prediksi urutan output. Performa model dapat ditingkatkan dengan penambahan **dropout**, **encoder bidirectional**, atau **attention mechanism** untuk fokus yang lebih baik pada input saat decoding.

In [4]:
# Hyperparameters
hidden_dim = 256
encoder = Encoder(src_vocab_size, hidden_dim)
decoder = Decoder(trg_vocab_size, hidden_dim)
model = Seq2Seq(encoder, decoder)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for src_batch, trg_batch in train_loader:
        optimizer.zero_grad()
        output = model(src_batch, trg_batch)
        loss = criterion(output.view(-1, trg_vocab_size), trg_batch.view(-1))
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 10/100, Loss: 0.49359074234962463
Epoch 20/100, Loss: 0.33720406889915466
Epoch 30/100, Loss: 0.32823798060417175
Epoch 40/100, Loss: 0.3265061676502228
Epoch 50/100, Loss: 0.32591694593429565
Epoch 60/100, Loss: 0.3256341218948364
Epoch 70/100, Loss: 0.32546094059944153
Epoch 80/100, Loss: 0.32533586025238037
Epoch 90/100, Loss: 0.32523614168167114
Epoch 100/100, Loss: 0.32515260577201843


In [6]:
# Function untuk menerjemahkan kalimat
def translate_sentence(model, sentence, src_vocab, trg_vocab):
    model.eval()
    tokens = [src_vocab[token] for token in sentence.split()] + [src_vocab['<eos>']]
    src_tensor = torch.tensor([tokens])

    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
        trg_indexes = [trg_vocab['<sos>']]

        for _ in range(10):  # Batas panjang kalimat hasil
            trg_tensor = torch.tensor([trg_indexes[-1]])
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
            pred_token = output.argmax(1).item()
            trg_indexes.append(pred_token)

            if pred_token == trg_vocab['<eos>']:
                break

    translated_sentence = [list(trg_vocab.keys())[list(trg_vocab.values()).index(idx)] for idx in trg_indexes[1:-1]]
    return ' '.join(translated_sentence)

# Contoh penggunaan
translated_output = translate_sentence(model, "Nice to meet you", src_vocab, trg_vocab)
print("Terjemahan:", translated_output)

KeyError: 'Nice'

# Nomor 1

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

for epoch in range(num_epochs):
    train_loss = 0
    model.train()
    for src_batch, trg_batch in train_loader:
        optimizer.zero_grad()
        output = model(src_batch, trg_batch)
        loss = criterion(output.view(-1, trg_vocab_size), trg_batch.view(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    scheduler.step(train_loss)
    print(f'Epoch {epoch + 1}, Loss: {train_loss}')

# Nomor 2

In [ ]:
# Model Encoder-Decoder dengan dropout opsional
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout=0.0):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, dropout=0.0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(1)
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size)

        hidden, cell = self.encoder(src)

        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t] = output

            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

# Hyperparameters
hidden_dim = 256
dropout_rate = 0.3  # Dropout rate untuk eksperimen
encoder_with_dropout = Encoder(src_vocab_size, hidden_dim, dropout=dropout_rate)
decoder_with_dropout = Decoder(trg_vocab_size, hidden_dim, dropout=dropout_rate)
model_with_dropout = Seq2Seq(encoder_with_dropout, decoder_with_dropout)

encoder_without_dropout = Encoder(src_vocab_size, hidden_dim)
decoder_without_dropout = Decoder(trg_vocab_size, hidden_dim)
model_without_dropout = Seq2Seq(encoder_without_dropout, decoder_without_dropout)

criterion = nn.CrossEntropyLoss()
optimizer_with_dropout = optim.Adam(model_with_dropout.parameters(), lr=0.001)
optimizer_without_dropout = optim.Adam(model_without_dropout.parameters(), lr=0.001)

# Training function
def train_model(model, optimizer, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        for src_batch, trg_batch in train_loader:
            optimizer.zero_grad()
            output = model(src_batch, trg_batch)
            loss = criterion(output.view(-1, trg_vocab_size), trg_batch.view(-1))
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Eksperimen dengan dropout
print("Training with dropout...")
train_model(model_with_dropout, optimizer_with_dropout)

# Eksperimen tanpa dropout
print("\nTraining without dropout...")
train_model(model_without_dropout, optimizer_without_dropout)

# Nomor 3

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1, bidirectional=True):
        super(Encoder, self).__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, batch_first=True)
        self.hidden_dim = hidden_dim
        self.num_directions = 2 if bidirectional else 1

    def forward(self, x):
        outputs, (hidden, cell) = self.rnn(x)
        if self.num_directions == 2:  # Jika bidirectional
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1).unsqueeze(0)
            cell = torch.cat((cell[-2], cell[-1]), dim=1).unsqueeze(0)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(1)
        output, (hidden, cell) = self.rnn(x, (hidden, cell))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden, cell

# Nomor 4

In [ ]:
# Fungsi pelatihan dengan scheduled sampling
def train_with_scheduled_sampling(model, train_loader, optimizer, criterion, trg_vocab_size, num_epochs=10, scheduled_sampling_ratio=0.5):
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0

        for src_batch, trg_batch in train_loader:
            optimizer.zero_grad()
            output = model(src_batch, trg_batch, scheduled_sampling_ratio)
            loss = criterion(output.view(-1, trg_vocab_size), trg_batch.view(-1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # Kurangi scheduled_sampling_ratio secara bertahap jika diinginkan
        if epoch % 5 == 0 and scheduled_sampling_ratio < 1.0:
            scheduled_sampling_ratio += 0.05

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader)}')

# Penyesuaian pada metode forward model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, scheduled_sampling_ratio=0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(src.device)

        hidden, cell = self.encoder(src)
        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t] = output

            top1 = output.argmax(1)
            use_teacher_forcing = random.random() < scheduled_sampling_ratio
            input_token = trg[:, t] if use_teacher_forcing else top1

        return outputs

# Panggil fungsi pelatihan dengan scheduled sampling
train_with_scheduled_sampling(model_with_dropout, train_loader, optimizer_with_dropout, criterion, trg_vocab_size, num_epochs=100)

# Nomor 5

In [ ]:
# Encoder menggunakan GRU
class EncoderGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1, bidirectional=False):
        super(EncoderGRU, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, num_layers, bidirectional=bidirectional, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, hidden = self.rnn(embedded)
        if self.rnn.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1).unsqueeze(0)
        return hidden

# Decoder menggunakan GRU
class DecoderGRU(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers=1):
        super(DecoderGRU, self).__init__()
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden):
        x = x.unsqueeze(1)
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden

# Model Seq2Seq dengan GRU
class Seq2SeqGRU(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqGRU, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(src.device)

        hidden = self.encoder(src)
        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input_token, hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

# Definisi hyperparameters dan model
hidden_dim = 256
encoder_gru = EncoderGRU(src_vocab_size, hidden_dim)
decoder_gru = DecoderGRU(trg_vocab_size, hidden_dim)
model_gru = Seq2SeqGRU(encoder_gru, decoder_gru)

criterion = nn.CrossEntropyLoss()
optimizer_gru = optim.Adam(model_gru.parameters(), lr=0.001)

# Fungsi pelatihan sama seperti sebelumnya
def train_model(model, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for src_batch, trg_batch in train_loader:
            optimizer.zero_grad()
            output = model(src_batch, trg_batch)
            loss = criterion(output.view(-1, trg_vocab_size), trg_batch.view(-1))
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Latihan model dengan GRU
train_model(model_gru, optimizer_gru)


# Nomor 6

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch

# Fungsi untuk melakukan prediksi dengan model
def translate(model, src_sentence, src_vocab, trg_vocab, max_len=20):
    model.eval()
    tokens = [src_vocab[token] for token in src_sentence.split()] + [src_vocab['<eos>']]
    src_tensor = torch.tensor([tokens]).to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        hidden = model.encoder(src_tensor)
        trg_indexes = [trg_vocab['<sos>']]

        for _ in range(max_len):
            trg_tensor = torch.tensor([trg_indexes[-1]]).to('cuda' if torch.cuda.is_available() else 'cpu')
            output, hidden = model.decoder(trg_tensor, hidden)
            pred_token = output.argmax(1).item()
            trg_indexes.append(pred_token)

            if pred_token == trg_vocab['<eos>']:
                break

    translated_sentence = [list(trg_vocab.keys())[list(trg_vocab.values()).index(idx)] for idx in trg_indexes[1:-1]]
    return ' '.join(translated_sentence)

# Evaluasi skor BLEU
def evaluate_bleu(model, src_sentences, trg_sentences, src_vocab, trg_vocab):
    bleu_scores = []
    for i, src_sentence in enumerate(src_sentences):
        reference = [trg_sentences[i].split()]  # Referensi target
        candidate = translate(model, src_sentence, src_vocab, trg_vocab).split()  # Prediksi model
        score = sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(score)
        print(f'Sentence {i + 1}: BLEU score = {score:.4f}')
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f'\nAverage BLEU score: {avg_bleu:.4f}')
    return avg_bleu

# Contoh data untuk evaluasi
src_sentences = ["How are you?"]  # Kalimat input
trg_sentences = ["comment vas tu?"]  # Terjemahan referensi

# Evaluasi model dengan skor BLEU
average_bleu = evaluate_bleu(model_gru, src_sentences, trg_sentences, src_vocab, trg_vocab)
